# Hologram manipulation

This notebook is intended to control a Spatial Light Modulator (SLM) in the Julia language.  It is compatible with the PLUTO VIS SLM (Holoeye $1920\times1080$ pixels), but can be easily adapted by changing the resolution (Ux, Uy, Hsize and Vsize) to any other phase--only SLM.

Requisites:
- Julia 1.0+
- Images package
- Holofuns.jl, Beams.jl and Holomember.jl
- screen
- geeqie
- vlc

## Load packages
First, lets load some useful libraries for the hologram creation

In [3]:
using Images
using Plots
include("/mnt/bendata/Documents/Docs/PhysicsLocal/HoloJulia/Holofuns.jl")
include("/mnt/bendata/Documents/Docs/PhysicsLocal/HoloJulia/Holomember.jl");  # this also loads beams.jl and MatrixTrans.jl

Lets define the numerical grid, considering the physical aspects of the Spatial Light Modulator

In [4]:
# SPATIAL SCALE
Ux=15.36E-3
Uy=8.64E-3
Hsize=div(1920,1); #1920 1440
Vsize=div(1080,1); #1080 900
 
# Generates ranges for xs and ys
ys = Ux*(2/Hsize)*collect(range(-Hsize/2,length=Hsize,stop=Hsize/2-1))
xs = Uy*(2/Vsize)*collect(range(-Vsize/2,length=Vsize,stop=Vsize/2-1))

# Coordinates
mx, ny = length(xs), length(ys)
Ys = reshape(xs, mx, 1)
Xs = reshape(ys, 1, ny);

## Beams
### Laguerre-Gaussian

In [11]:
w0 = 2E-3
l = 1
p = 0
phi0 = 0.0
@time U = LaguerreGaussBeam.(Xs, Ys, w0, phi0, l, p);

  0.166687 seconds (31 allocations: 31.641 MiB, 0.63% gc time)


### Hermite-Gaussian

In [409]:
w0 = 2E-3
m = 2
n = 2
phi0 = 0.0
@time U = HermiteGaussBeam.(Xs, Ys, w0, phi0, m, n);

  0.259786 seconds (31 allocations: 31.641 MiB, 0.73% gc time)


### Ince-Gaussian

In [151]:
# Ince-Gaussian parameters
p = 6    # x-direction
m = 4    # y-direction
q = 1.5
kind = 2
phi0 = 0.0

# Pre-calculates Ince coefficients
if kind == 0
    Cs = CInceCoef(p,m,q)
elseif kind == 1
    Ss = SInceCoef(p,m,q)
else
    Cs = CInceCoef(p,m,q)
    Ss = SInceCoef(p,m,q)
end

if kind == 0 # IG even
    IGE = IGBeamE.(Xs, Ys , w0, phi0, p, m, q, (Cs,))
    #NormN = IGE[abs2.(IGE).==maximum(abs2.(IGE))]
    #NormP = IGE[angle.(IGE).==maximum(angle.(IGE))]
    V = IGE

elseif kind == 1 # IG odd
    IGO = IGBeamO.(Xs, Ys , w0, phi0, p, m, q, (Ss,))
    #NormN = IGO[abs2.(IGO).==maximum(abs2.(IGO))]
    #NormP = IGE[angle.(IGE).==maximum(angle.(IGE))]
    V = IGO

else # kind == 2 # IG helical
    IGE = IGBeamE.(Xs, Ys , w0, phi0, p, m, q, (Cs,))
    NormE = IGE[abs2.(IGE).==maximum(abs2.(IGE))]
    IGE = IGE/NormE[1]
    IGO = IGBeamO.(Xs, Ys , w0, phi0, p, m, q, (Ss,))
    NormO = IGO[abs2.(IGO).==maximum(abs2.(IGO))]
    IGO = IGO/NormO[1]
    V = IGE+im*IGO
    #Norm = Uaux[abs2.(Uaux).==maximum(abs2.(Uaux))]
    #Uaux = Uaux/Norm[1]
end;

## Hologram creation (static)
With the generated field in the previous section, we will create the hologram and display it in the SLM screen

In [12]:
# Hologram parameters
lines = 150   # amount of lines in the whole screen
alf = 0

# Hologram creation
Hologram = HoloCrea(U, lines, alf)

# Position of the hologram -- depends on the screen resolution of the user display
screenX=div(1600,1)
screenY=1
dirname = "/mnt/bendata/Documents/Docs/PhysicsLocal/HoloJulia/tmp"
save("$dirname/tmp.png", colorview(Gray, Hologram))

# Command line code
# run(`screen -d -m geeqie`)   # dirty!!!
# run(`killall geeqie`)        # dirty!!!
run(`screen -d -m geeqie --geometry=$(Hsize)x$(Vsize)+$(screenX)+$(screenY) -f $dirname/tmp.png`);

┌ Info: Recompiling stale cache file /home/ben/.julia/compiled/v1.0/ImageMagick/0LbNX.ji for ImageMagick [6218d12a-5da1-5696-b52f-db25d2ecc6d1]
└ @ Base loading.jl:1190


## Partially coherent beams
In this section we will generate holograms to simulate a partially coherent structured beam.  Our description of a spatially partially coherent source can be seen as the averaging process of randomly located beams within a circular region.  The size of the circular region is related to the spatial coherence of the source.

We can use the funcion the functions
- memberPCLG - Laguerre-Gaussian
- memberPCLGmt - Laguerre-Gaussian (with multithreading support)
- memberPCHG - Hermite-Gaussian
- memberPCHGmt - Hermite-Gaussian (with multithreading support)
- memberPCIG - Ince-Gaussian
- memberPCIGmt - Ince-Gaussian (with multithreading support)

Note that the multithreading support in Julia is still experimental and might do some weird stuff.

In [24]:
function ensemblePC(Useed, Vsize, Hsize, circle, N, Ne, lines, alf, dirname, filename)
    
    # Generates the desired holograms according to the given parameters
    for memberE = 1:Ne
        # Computes the field of one member
        Upc = memberPCmt(Useed, Vsize, Hsize, circle, N)

        # Hologram creation
        Hologram = HoloCrea(Upc, lines, alf)
        save("$dirname/$(filename)-$(lpad(memberE,3,'0')).png", colorview(Gray, Hologram))
    end
    
end

ensemblePC (generic function with 1 method)

### Partially coherent Laguerre--Gaussian

First, lets define the beam parameters and the seed beam to generate its partially coherent version

In [21]:
# Beam parameters
w0 = 2E-3
l = 1
p = 0

# Seed beam
Useed = LaguerreGaussBeam.(Xs, Ys, w0, 0.0, l, p);

In [27]:
# Statistical parameters
circle = w0 * 0.9
N = 100
Ne = 2

# Hologram parameters
lines = 100   # amount of lines in the whole screen
alf = 0
dirnameLG = "/mnt/bendata/Documents/Docs/PhysicsLocal/HoloJulia/tmp/pc/LG"
filenameLG = "LG-$(l)$(p)-$(circle/w0)"

@time ensemblePC(Useed, Vsize, Hsize, circle, N, Ne, lines, alf, dirnameLG, filenameLG)

  9.515536 seconds (4.09 k allocations: 33.119 GiB, 19.55% gc time)


### Partially coherent Hermite--Gaussian

In [53]:
# Beam parameters
w0 = 2E-3
m = 2
n = 4
Useed = HermiteGaussBeam.(Xs, Ys, w0, 0.0, m, n);

In [54]:
# Statistical parameters
circle = w0 * 0.9
N = 200
Ne = 60

# Hologram parameters
lines = 100   # amount of lines in the whole screen
alf = 0
dirnameHG = "/mnt/bendata/Documents/Docs/PhysicsLocal/HoloJulia/tmp/pc/HG"
filenameHG = "HG-$(m)$(n)-$(circle/w0)"

# Hologram creation per member
@time ensemblePC(Useed, Vsize, Hsize, circle, N, Ne, lines, alf, dirnameHG, filenameHG)

577.098496 seconds (216.04 k allocations: 1.940 TiB, 22.22% gc time)


### Partially coherent Ince--Gaussian

In [56]:
# Beam parameters
w0 = 2E-3
p = 3
m = 3
q = 1.5
kind = 0

# Pre-calculates Ince coefficients
if kind == 0
    Cs = CInceCoef(p,m,q)
elseif kind == 1
    Ss = SInceCoef(p,m,q)
else
    Cs = CInceCoef(p,m,q)
    Ss = SInceCoef(p,m,q)
end

# Seed beam!  IG beams are funny!  @_@
if kind == 0 # IG even
    IGE = IGBeamE.(Xs, Ys, w0, 0.0, p, m, q, (Cs,))
    #NormN = IGE[abs2.(IGE).==maximum(abs2.(IGE))]
    #NormP = IGE[angle.(IGE).==maximum(angle.(IGE))]
    Useed = IGE

elseif kind == 1 # IG odd
    IGO = IGBeamO.(Xs, Ys, w0, 0.0, p, m, q, (Ss,))
    #NormN = IGO[abs2.(IGO).==maximum(abs2.(IGO))]
    #NormP = IGE[angle.(IGE).==maximum(angle.(IGE))]
    Useed = IGO

else # kind == 2 # IG helical
    IGE = IGBeamE.(Xs, Ys, w0, 0.0, p, m, q, (Cs,))
    NormE = IGE[abs2.(IGE).==maximum(abs2.(IGE))]
    IGE = IGE/NormE[1]
    IGO = IGBeamO.(Xs, Ys, w0, 0.0, p, m, q, (Ss,))
    NormO = IGO[abs2.(IGO).==maximum(abs2.(IGO))]
    IGO = IGO/NormO[1]
    Useed = IGE+im*IGO
    #Norm = Uaux[abs2.(Useed).==maximum(abs2.(Useed))]
    #Useed = Uaux/Norm[1]
end;

In [57]:
# Statistical parameters
circle = w0 * 0.9
N = 200
Ne = 180

# Hologram parameters
lines = 100   # amount of lines in the whole screen
alf = 0
dirnameIG = "/mnt/bendata/Documents/Docs/PhysicsLocal/HoloJulia/tmp/pc/IG"
filenameIG = "IG-$(p)$(m)$(q)-$(kind)-$(circle/w0)"

@time ensemblePC(Useed, Vsize, Hsize, circle, N, Ne, lines, alf, dirnameIG, filenameIG)

1672.998978 seconds (647.67 k allocations: 5.815 TiB, 22.33% gc time)


## Hologram creation (dynamic)

With the created holograms from the previous section, we generate a video (dynamic hologram) and then we display it in the SLM 

In [58]:
# Position of the hologram -- depends on the screen resolution of the user display
screenX = div(1600,1)
screenY = 1

# Code to generate video and run it in VLC\
BM = "IG"
if BM == "LG"
    run(`ffmpeg -framerate 60 -i $dirnameLG/LG-$(l)$(p)-$(circle/w0)-%03d.png -c:v libx264 -pix_fmt yuv420p -qp 0 $dirnameLG/LG-$(l)$(p)-$(circle/w0)-$(N)-$(Ne).mp4`);
    run(`screen -d -m mplayer -vo x11 -geometry $(screenX) -zoom -fs -loop 0 $dirnameLG/LG-$(l)$(p)-$(circle/w0)-$(N)-$(Ne).mp4`)
    
elseif BM == "HG"
    run(`ffmpeg -framerate 60 -i $dirnameHG/HG-$(m)$(n)-$(circle/w0)-%03d.png -c:v libx264 -pix_fmt yuv420p -qp 0 $dirnameHG/HG-$(m)$(n)-$(circle/w0)-$(N)-$(Ne).mp4`);
    run(`screen -d -m mplayer -vo x11 -geometry $(screenX) -zoom -fs -loop 0 $dirnameHG/HG-$(m)$(n)-$(circle/w0)-$(N)-$(Ne).mp4`)
    
else
    run(`ffmpeg -framerate 15 -i $dirnameIG/IG-$(p)$(m)$(q)-$(kind)-$(circle/w0)-%03d.png -c:v libx264 -pix_fmt yuv420p -qp 0 $dirnameIG/IG-$(p)$(m)$(q)-$(kind)-$(circle/w0)-$(N)-$(Ne).mp4`);
    run(`screen -d -m mplayer -vo x11 -geometry $(screenX) -zoom -fs -loop 0 $dirnameIG/IG-$(p)$(m)$(q)-$(kind)-$(circle/w0)-$(N)-$(Ne).mp4`)
end

ffmpeg version n4.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 8.2.1 (GCC) 20180831
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxml2 --enable-libxvid --enable-nvdec --enable-nvenc --enable-omx --enable-shared --enable-version3
  libavutil      56. 22.100 / 56. 22.100
  libavcodec     58. 35.100 / 58. 35.100
  libavformat    58.

Process(`screen -d -m mplayer -vo x11 -geometry 1600 -zoom -fs -loop 0 /mnt/bendata/Documents/Docs/PhysicsLocal/HoloJulia/tmp/pc/IG/IG-331.5-0-0.9-200-180.mp4`, ProcessExited(0))

In [31]:
Threads.nthreads()

4